# Download data for noise
This part of the code will be used to download noise data.

It is based on Josh Russell's **fetch_NOISE**.

*william b hawley april 2020*

In [1]:
from config import *
import os
import sys

import numpy as np
import matplotlib.pyplot as plt

import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from obspy.geodetics import gps2dist_azimuth, locations2degrees
from obspy.io.sac import SACTrace

from datetime import datetime
import calendar

%matplotlib inline
%load_ext autoreload
%autoreload

In [2]:
# Check to see if data directory exists
if not os.path.exists(NoiseDataDir):
    os.makedirs(NoiseDataDir)

# load the client
client = Client(webservice)

In [3]:
# Load stations

t1 = UTCDateTime(tstart)
t1 = t1 - (60*60*24)*noDays
t2 = UTCDateTime(tend)

inventory = client.get_stations(network=network, station=','.join(StaList), channel=','.join(ChanList), starttime=t1, endtime=t2)

In [4]:
# Create file with days to download

# load event file
ifn = open(EventsFileName)
DayList = []
with open(DayFileName,'w') as f:
    for line in ifn:
        evDate = UTCDateTime(line)
        if isCalDay == 1:
            evDate = UTCDateTime(evDate.year,evDate.month,evDate.day)
        for iday in range(1,noDays+1):
            date = evDate-(60*60*24)*iday
            date = datetime.strptime(str(date),'%Y-%m-%dT%H:%M:%S.%fZ')
            date = date.strftime('%Y%m%d%H%M')
            f.write(date+'\n')
            DayList.append(date)
f.close()
print("Downloading noise for "+str(len(DayList))+" days.")

In [5]:
# Download data

# get stations with BXH channel
XStationList = open(XStafn)

# loop through stations
for ista in range(0,len(inventory[0])):
    
    # get station info
    stel = inventory[0].stations[ista].elevation
    stla = inventory[0].stations[ista].latitude
    stlo = inventory[0].stations[ista].longitude
    stnm = inventory[0].stations[ista].code
    # make station file
    print("Working on station "+stnm+"...")
    staDir = NoiseDataDir + network + '_' + stnm + '/'
    if not os.path.exists(staDir):
        os.makedirs(staDir)
    
    # loop through days in noise day file
    for iday in DayList:
        time1 = UTCDateTime(iday)
        year = str(time1.year)
        jday = str("{:03d}".format(time1.julday))
        hh = str("{:02d}".format(time1.hour))
        mm = str("{:02d}".format(time1.minute))
        ss = str("{:02d}".format(time1.second))
        time2 = time1 + (60*60*24)
        
        # loop through channels
        for comp in ChanList:
            # if a BXH exists, use that.
            if comp == 'BDH':
                if stnm in XStationList:
                    comp = 'BXH'
                    #print("Using comp "+comp+" for station "+stnm)
            # download
            sac_out = staDir + stnm + '.' + year + '.' + jday + '.' + hh + '.' + mm + '.' + ss + '.' + comp + '.sac'
            if os.path.exists(sac_out):
                print(sac_out+" already exists, skipping...")
                continue
            try:
                st = client.get_waveforms(network=network, station=stnm, location='*', channel=comp, starttime=time1, endtime=time2, attach_response=True)
            except Exception:
                print('Missing data for station: ',stnm)
                continue
            
            # if BXH, change back ... doesn't matter; we downsample to 1 Hz anyway
            if comp == 'BXH':
                comp = 'BDH'

            sr = st[0].stats.sampling_rate
            
            # check for gaps
            if len(st) > 1:
                st.merge(method=1, fill_value=0)
            
            # remove response
            if isRemoveResp:
                try:
                    st.remove_response(output="DISP", zero_mean=True, taper=True, taper_fraction=0.05, pre_filt=[0.001, 0.005, sr/3, sr/2], water_level=60)
                except Exception:
                    print('Failed to remove response: ' + iday)
                    continue

            st.trim(starttime=time1, endtime=time2, pad=True, nearest_sample=False, fill_value=0) # make sure correct length
            st.detrend(type='demean')
            st.detrend(type='linear')
            st.taper(type="cosine",max_percentage=0.05)
            
            # downsample
            if isDownsamp and sr!=srNew:
                st.filter('lowpass', freq=0.4*srNew, zerophase=True) # anti-alias filter
                st.decimate(factor=int(sr/srNew), no_filter=True) # downsample
                st.detrend(type='demean')
                st.detrend(type='linear')
                st.taper(type="cosine",max_percentage=0.05)

            # convert to SAC and fill out station/event header info
            sac = SACTrace.from_obspy_trace(st[0])
            sac.stel = stel
            sac.stla = stla
            sac.stlo = stlo
            sac.kcmpnm = comp
            sac.delta = (1/srNew)
            
            # write sac file
            sac_out = staDir + stnm + '.' + year + '.' + jday + '.' + hh + '.' + mm + '.' + ss + '.' + comp + '.sac'
            sac.write(sac_out)
XStationList.close()

Working on station BB030...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.270.00.00.00.LHZ.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.270.00.00.00.LH1.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.270.00.00.00.LH2.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.270.00.00.00.BDH.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.267.00.00.00.LHZ.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.267.00.00.00.LH1.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.267.00.00.00.LH2.sac already exists, skipping...
/Users/whawley/Research/github/Blanco-SW/data/SAC_Noise/X9_BB030/BB030.2013.267.00.00.00.BDH.sac 